In [3]:
import pandas as pd

df = pd.read_csv('water-pollution-data.csv')

/var/folders/rq/fttfpph53671nf5_9s4qnwlm0000gn/T/ipykernel_7179/1095172746.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('water-pollution-data.csv')


In [4]:
df.head()

,_id,station_id,station_name,station_number,full_station_name,station_type,latitude,longitude,status,county_name,...,sample_depth_units,anl_data_type,parameter,fdr_result,fdr_text_result,fdr_date_result,fdr_reporting_limit,uns_name,mth_name,fdr_footnote
0,1,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,...,Feet,NaN,DissolvedOxygen,9.2,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN
1,2,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,...,Feet,NaN,ElectricalConductance,515.0,NaN,NaN,1.0,uS/cm,Std Method 2510-B (Field),NaN
2,3,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,...,Feet,NaN,WaterTemperature,6.7,NaN,NaN,0.1,°C,EPA 170.1 (Field),NaN
3,4,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,...,Feet,NaN,pH,7.3,NaN,NaN,0.1,pH Units,EPA 150.1 (Field),NaN
4,5,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,...,Feet,NaN,DissolvedOxygen,9.7,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN


In [11]:
filtered_df_2016 = df[df['sample_date'].str.contains("2016")]

In [13]:
filtered_df_2016_DO = filtered_df_2016[filtered_df_2016['parameter'].str.contains("DissolvedOxygen")]

In [25]:
filtered_df_2016_DO_new = filtered_df_2016_DO[['county_name','parameter', 'fdr_result']]

In [32]:
filtered_df_2016_DO_new.groupby('county_name').mean('fdr_result').sort_values(by=['fdr_result'])

,fdr_result
county_name,
San Mateo,2.770000
Fresno,4.626000
Mendocino,5.824118
Stanislaus,5.886129
Lassen,6.646154
Sutter,6.743235
Sonoma,6.873500
Merced,7.030137
Lake,7.203934


In [51]:
counties = pd.read_csv('CountyGeoID.csv',converters={'GeoID': str})

In [52]:
counties.head()

,County,GeoID
0,Alameda County,06001
1,Contra Costa County,06013
2,Del Norte County,06015
3,Humboldt County,06023
4,Lake County,06033


In [61]:
def construct_url(row):
    base_url = 'https://coast.noaa.gov/ccapatlas/api/v1/landcover?StartDate=1996&EndDate=2016'
    parameter = row['GeoID']  
    url = f'{base_url}&GeoId={parameter}'
    return url

# Apply the function to each row of the DataFrame
counties['URL'] = counties.apply(construct_url, axis=1)

In [71]:
for index, row in counties.iterrows():
    url = row['URL']
    data = pd.read_json(url)
    df = pd.DataFrame(data)
    report_data = df['getTotalAndChangePercentagesForReport'][0]
    forest_data = pd.DataFrame(report_data)
    row['forestCover'] = forest_data['forest'][0]['total'] 
    print(row['County'],row['forestCover'])

Alameda County 16.96
Contra Costa County 15.13
Del Norte County 64.80
Humboldt County  67.24
Lake County 30.07
Los Angeles County 5.82
Marin County 20.99
Mendocino County 59.15
Monterey County 18.12
Napa County 27.29
Orange County 2.61
Sacramento County 1.19
San Benito County 13.78
San Diego County 3.63
San Francisco County 0.95
San Joaquin County 1.03
San Luis Obispo County 13.25
San Mateo County 30.68
Santa Barbara County 12.82
Santa Clara County 32.46
Santa Cruz County 50.23
Solano County 2.65
Sonoma County 37.96
Sutter County 1.61
Trinity County 67.89
Ventura County 14.93
Yolo County 3.74


In [70]:
counties.head()

,County,GeoID,URL,forestCover
0,Alameda County,06001,https://coast.noaa.gov/ccapatlas/api/v1/landco...,16.96
1,Contra Costa County,06013,https://coast.noaa.gov/ccapatlas/api/v1/landco...,15.13
2,Del Norte County,06015,https://coast.noaa.gov/ccapatlas/api/v1/landco...,64.80
3,Humboldt County,06023,https://coast.noaa.gov/ccapatlas/api/v1/landco...,67.24
4,Lake County,06033,https://coast.noaa.gov/ccapatlas/api/v1/landco...,30.07


In [19]:
df = pd.DataFrame(data)

In [20]:
df.head()

,getCountyOverallChangePercent,getCountyOverallDistributionOfLandCoverByType,getCountyDistributionLossesGains,getLandCoverChangeSummaryTableForReporting,getCountyDevelopedAreaPercentage,getCountyImperviousAreaPercentage,getCountyDevelopedAreaPercentageChange,getCountyImpervAreaPercentageChange,getCountyDistributionNetDevelopedChange,getCountyDistributionLossByClass,...,getCountyWetlandSaltwaterAreaChange,getCountyWetlandsChange,getCountyWetlandsDistributionLossesByClass,getPercentageHiMedLowOpenVsNonDeveloped,getPercentageForestedVsNonForested,getNetChangeForestFragmentation,getPercentageWetlandsVsNonWetlands,getNetChangeAllWetlandsClasses,getNetChangeGainedOrLostFromNonWetland,getTotalAndChangePercentagesForReport
LandCoverAtlas,[{'value': '2.90'}],"[{'categoryAbbr': ['HID', 'LID', 'OSD', 'GRS',...","[{'categoryAbbr': ['HID', 'LID', 'OSD', 'GRS',...","[{'Category': 'Developed, High Intensity (HID)...","[{'StartDate': '28.82', 'EndDate': '29.95'}]","[{'StartDate': '14.29', 'EndDate': '14.85'}]",[{'value': '3.91'}],[{'value': '3.95'}],"[{'categoryAbbr': ['HID', 'MID', 'LID', 'OSD']...","[{'categoryAbbr': ['AGR', 'BAR', 'EMW', 'FOR',...",...,[{'value': '17.87'}],"[{'categoryAbbr': ['PFW', 'PSW', 'PEW', 'EFW',...","[{'categoryAbbr': ['HID', 'LID', 'OSD', 'BAR',...","[{'data': [{'name': 'Open Space', 'y': 3.9}, {...","[{'data': [{'name': 'Non Forest', 'y': 83}, {'...","[{'categoryAbbr': [], 'category': ['Deciduous'...","[{'data': [{'name': 'Non Wetlands', 'y': 97.1}...","[{'categoryAbbr': [], 'category': ['Palustrine...","[{'categoryAbbr': ['DEV', 'AGR', 'BAR', 'WTR']...","[{'development': {'impervious': '14.85', 'impe..."


In [28]:
df['getTotalAndChangePercentagesForReport']

LandCoverAtlas    [{'development': {'impervious': '14.85', 'impe...
Name: getTotalAndChangePercentagesForReport, dtype: object

In [32]:
report_data = df['getTotalAndChangePercentagesForReport'][0]

In [33]:
forest_data = pd.DataFrame(report_data)

In [39]:
print(forest_data['forest'][0]['total'])

16.96
